## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from G_key import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('WeatherPy_Database.csv')
city_data_df.head()
    

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lompoc,US,34.6391,-120.4579,68.02,65,0,17.00,clear sky
1,1,Hobyo,SO,5.3505,48.5268,83.77,80,44,14.27,scattered clouds
2,2,Chernoyerkovskaya,RU,45.4331,37.7686,53.24,77,73,5.91,broken clouds
3,3,Busselton,AU,-33.6500,115.3333,61.48,87,94,5.08,overcast clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,46.06,71,75,4.61,broken clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Please enter minimun temperature requirements '))
max_temp = float(input('Please enter maximun temperature requirements '))

Please enter minimun temperature requirements 40
Please enter maximun temperature requirements 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_location_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & 
                                      (city_data_df['Max Temp'] >= min_temp)]

preferred_location_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lompoc,US,34.6391,-120.4579,68.02,65,0,17.00,clear sky
1,1,Hobyo,SO,5.3505,48.5268,83.77,80,44,14.27,scattered clouds
2,2,Chernoyerkovskaya,RU,45.4331,37.7686,53.24,77,73,5.91,broken clouds
3,3,Busselton,AU,-33.6500,115.3333,61.48,87,94,5.08,overcast clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,46.06,71,75,4.61,broken clouds


In [16]:
# 4a. Determine if there are any empty rows.
preferred_location_df.isnull().sum() 

City_ID                0
City                   0
Country                7
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
import numpy as np

In [20]:
preferred_location_df['Country'].replace('', np.nan, inplace=True)
preferred_location_df.dropna(subset=['Country'], inplace=True)
preferred_location_df.isnull().sum()

C:\Users\tavio\anaconda3\envs\Pythondata\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\tavio\anaconda3\envs\Pythondata\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_location_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lompoc,US,68.02,clear sky,34.6391,-120.4579,
1,Hobyo,SO,83.77,scattered clouds,5.3505,48.5268,
2,Chernoyerkovskaya,RU,53.24,broken clouds,45.4331,37.7686,
3,Busselton,AU,61.48,overcast clouds,-33.6500,115.3333,
4,Ushuaia,AR,46.06,broken clouds,-54.8000,-68.3000,
5,Fomboni,KM,83.46,few clouds,-12.2800,43.7425,
6,Dawei,MM,76.69,overcast clouds,14.0833,98.2000,
7,Hobart,AU,53.26,overcast clouds,-42.8794,147.3294,
8,Hithadhoo,MV,83.46,overcast clouds,-0.6000,73.0833,
9,Arraial Do Cabo,BR,76.86,scattered clouds,-22.9661,-42.0278,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except(IndexError):
        print('No hotel found')
       
        

No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found
No hotel found


In [37]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [38]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lompoc,US,68.02,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
1,Hobyo,SO,83.77,scattered clouds,5.3505,48.5268,Iftin hotel hobyo
2,Chernoyerkovskaya,RU,53.24,broken clouds,45.4331,37.7686,Chernoyerkovskaya
3,Busselton,AU,61.48,overcast clouds,-33.6500,115.3333,Observatory Guest House
4,Ushuaia,AR,46.06,broken clouds,-54.8000,-68.3000,Albatross Hotel
...,...,...,...,...,...,...,...
682,Sweetwater,US,90.00,broken clouds,25.7634,-80.3731,Courtyard by Marriott Miami at Dolphin Mall
684,Basco,PH,77.09,broken clouds,20.4487,121.9702,Nathaniel's Lodge
686,Vlagtwedde,NL,59.25,clear sky,53.0275,7.1083,Roompot Vakantiepark Emslandermeer
687,Konya,TR,53.65,clear sky,38.1667,32.5000,Bahcesaray Koyu Muhtarligi


In [39]:
import csv
# 8a. Create the output File (CSV)
output_data_file = ("C:\\Users\\tavio\\Documents\\Data_Camp\\Analysis Folder\\World_Weather_Analysis\\Vacation_Search\\WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} </dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [1]:
# 11a. Add a marker layer for each city to the map. 
# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

NameError: name 'hotel_df' is not defined